# 7. 데이터 준비하기: 다듬기, 변형, 병합

- 데이터 분석과 모델링 작업은 데이터를 불러오고, 다듬고, 변형하고 재정렬하는, 데이터 준비 과정에 많은 시간이 소요
- 가끔 파일이나 데이터베이스에서 저장된 데이터가 애플리케이션에서 사용하기 쉽지 않은 방식으로 저장되어 있을 때도 존재
- 데이터가 저장된 형태를 다른 형태로 바꾸기 위해(파이썬, Perl, R, 자바 혹은 awk나 sed 같은 유닉스의 텍스트 처리 유틸리티도 사용하지만 파이썬 표준 라이브러리와 pandas를 함께 사용하면 큰 어려움 없이 데이터를 원하는 형태로 가공 가능
- pandas는 이런 작업을 위한 유연하고 빠른 고수준의 알고리즘과 처리 기능 제공
- pandas에서 찾을 수 없는 새로운 형태의 데이터 처리 방식을 발견하게 되면 알려달라고 함
- pandas는 대부분의 설계와 구현에 실제 애플리케이션 개발 과정 중에 발생한 요구사항 반영

## 7.1 데이터 합치기

### pandas 객체에 저장된 데이터는 여러 내장 함수를 이용해 합치기 가능

- pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터 베이스의 join 연산과 유사
- pandas.concat은 하나의 축을 따라 객체를 이어붙임
- combine_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

#### 내장 함수는 앞으로 이 책 전반에 걸쳐 계속 사용하게 될 것이므로 빠른 시간 내에 익숙해지는 것이 좋다

### 7.1.1 데이터베이스 스타일로 DataFrame 합치기

- merge나 join 연산은 관계형 데이터베이스의 핵심적인 연산으로, 키를 하나 이상 사용해서 데이터 집합의 로우를 합침

In [545]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

In [546]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

In [547]:
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})

In [548]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [549]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


#### 일대다의 예제

- df1 데이터는 key 칼럼에 여러 개의 a,b 존재
- df2의 kye 칼럼은 유일한 로우가 존재
- 이 객체에 대해 merge 함수 호출

In [550]:
#겹치는 key를 기준으로 합병
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [551]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


#### 두 객체에 공통되는 칼럼 이름이 하나도 없다면 따로 지정

-----

- df3는 칼럼이 lkey와 data1
- df4는 칼럼이 rkey와 data2
- 서로 겹치는게 하나도 없다. 
- 그러니 명시적으로 지정해줘야 한다.

------

In [552]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

In [553]:
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})

In [554]:
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [555]:
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [556]:
# left, right 둘 다 같이 써줘야 한다.
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [557]:
# TypeError! 
# right_on은 명시되지 않았으니 NoneType으로 넘어가서 비교가 되지 않는다.
pd.merge(df3, df4, left_on='lkey')

TypeError: object of type 'NoneType' has no len()

- 앞의 결과를 잘 살펴보면 'c'와 'd'에 해당하는 값이 빠짐
- merge 함수는 기본적으로 내부조인(inner join)을 수행하여 교집합인 결과를 반환

#### how keyword(조인 방법)

- 'left': 왼쪽 우선 외부조인. 왼쪽의 모든 로우를 포함하는 결과 반환
- 'right': 오른쪽 우선 외부조인. 오른쪽의 모든 로우를 포함하는 결과 반환
- 'outer': 완전 외부조인. 합집합인 결과 반환

In [600]:
#df1과 df2를 외부조인
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [601]:
# left에만 있는 c까지 포함이 된 것을 확인할 수 있다.
pd.merge(df1, df2, how='left')

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [602]:
# right에만 있는 d까지 포함
pd.merge(df1, df2, how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


#### 다 대 다 병합은 잘 정의되어 있긴 하지만 직관적이지는 않다.

In [603]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})

In [604]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})

In [605]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [606]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [607]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


#### 다 대 다 조인은 두 로우의 [데카르트 곱](http://whiteship.tistory.com/1365) 반환(b)

- 왼쪽 외부조인을 기준
- 왼쪽에 있는 b는 0,1,5 3개 존재
- 오른쪽에 있는 b는 1,3 2개 존재
- 0,1,5를 기준으로 2개씩 조합. 3 * 2 = 6. 총 6개의 결과
- 이 조인 메서드는 결과에 나타나는 구별되는 키에 대해서만 적용

In [608]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [609]:
# how 매개변수의 default 값은 inner이다.
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [610]:
pd.merge?

#### how Parameter

how : {'left', 'right', 'outer', 'inner'}, default 'inner'

- left: use only keys from left frame (SQL: left outer join)
- right: use only keys from right frame (SQL: right outer join)
- outer: use union of keys from both frames (SQL: full outer join)
- inner: use intersection of keys from both frames (SQL: inner join)

#### 여러 개의 키를 병합하려면 키의 칼럼 이름이 들어간 리스트를 넘김

In [611]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})

In [612]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})

In [613]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [614]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [615]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


------

#### outer 해석

- on parameter로 key1, key2를 기준으로 병합
- left에서 key1=foo, key2=one 일 때 lval은 1이다.
- right에서는 rval이 4이다.
- 이렇게 1개의 로우가 완성된다.
- 모든 로우에 대해서 실행하고
- 2, 4 인덱스처럼 한 쪽에만 값이 있는 경우는 NaN으로 표시

-----

#### merge 메서드의 종류에 따라 어떤 키 조합이 결과로 반환되는지 알려면 실제 구현과는 조금 다르지만 여러 개의 키가 들어있는 튜플의 배열이 단일 조인 키로 사용된다고 생각하면 된다.

#### 칼럼과 칼럼을 조인할 때 전달한 DataFrame 객체의 색인은 무시된다.

#### 머지 연산에서 고려해야 할 사항

- 겹치는 칼럼 이름에 대한 처리
- 축의 이름을 변경해서 수동으로 칼럼 이름을 겹치게 할 수도 있고
- merge 함수에 있는 suffixes 인자를 통해 두 DataFrame 객체에서 겹치는 칼럼 이름 뒤에 붙인 문자열을 지정할 수도 있음

In [616]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [617]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### merge 함수 인자 목록

인자 | 설명
--- | ---
left | 머지하려는 DataFrame 중 왼쪽에 위치한 DataFrame
right | 머지하려는 DataFrame 중 오른쪽에 위치한 DataFrame
how | 조인방법. 'inner', 'outer', 'left', 'right'. 기본값은 inner
on | 조인하려는 로우 이름. 반드시 두 DataFrame 객체 모두에 있는 이름이어야 한다. 만약 명시되지 않고 다른 조인 키도 주어지지 않으면 left와 right에서 공통되는 칼럼을 조인 키로 사용한다.
left_on | 조인 키로 사용할 left DataFrame의 칼럼
right_on | 조인 키로 사용할 right DataFrame의 칼럼
left_index | 조인 키로 사용할 left DataFrame의 색인 로우(다중 색인일 경우의 키)
right_index | 조인 키로 사용할 right DataFrame의 색인 로우(다중 색인일 경우의 키)
sort | 조인 키에 따라 병합된 데이터를 사전 순으로 정렬. 기본값은 True. 대용량 데이터의 경우 False라면 성능상의 이득을 얻을 수도 있다.
suffixes | 칼럼 이름이 겹칠 경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플. 기본값은 ('_x', '_y'). 만약 'data'라는 칼럼 이름이 양쪽 DataFrame에 같이 존재하면 결과에서는 'data_x', 'data_y'로 나타난다.
copy | False일 경우 예외적인 경우에 결과로 데이터가 복사되지 않도록 한다. 기본값은 항상 복사가 이루어진다.

### 7.1.2 색인 머지하기

- 머지하려는 키가 DataFrame의 색인일 수 있다.
- 이때에는 left_index = True, right_index = True 옵션을 지정해 해당 색인을 머지 키로 사용할 수 있다.

In [618]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                   'value': range(6)})

In [619]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [620]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [621]:
right1

,group_val
a,3.5
b,7.0


In [622]:
# left_on으로 어떤 컬럼이 조인 키로 사용될지 결정
# right_index로 어떤 인덱스가 조인 키로 사용될지 결정
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


-------

#### 해석

- left의 key를 기준으로 left1, right1을 머지하는데 right는 index를 기준으로 하겠다.
- left1에 a가 3개 존재
- right1에 a가 존재
- 머지된 값을 보면 양쪽에 모두 있는 a를 기준으로 합쳐진 것을 볼 수 있다.
- 다시 한 번 언급하지만 left1의 key와 right1의 right_index가 겹쳐지기 때문에 가능한 상황이다.
- 다른 옵션이 없기 때문에(default: inner. 교차) left1에 있는 c는 보여주지 않는다.

------

In [623]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [624]:
# right_on을 입력하지 않았기 때문에 error가 난다.
pd.merge(left1, right1, left_on='key')

TypeError: object of type 'NoneType' has no len()

In [665]:
# right_on에 group_val을 입력하면 교차하는게 하나도 없으니 아무것도 표시하지 않음
pd.merge(left1, right1, left_on='key', right_on='group_val')

,key,value,group_val


In [666]:
pd.merge(left1, right1, left_on='key', right_on='group_val', how='outer')

,key,value,group_val
0,a,0.0,NaN
1,a,2.0,NaN
2,a,3.0,NaN
3,b,1.0,NaN
4,b,4.0,NaN
5,c,5.0,NaN
6,NaN,NaN,3.5
7,NaN,NaN,7.0


#### 하나도 겹치는게 없을때 how로 outer

- 겹치는게 하나도 없으니 inner 조인은 아무것도 안나오는 반면 outer 조인은 모두 하나 이상은 NaN인 것을 확인할 수 있다.

#### 머지는 기본적으로 교집합을 구하지만 외부조인을 실행해서 합집합을 구할 수도 있다.

In [667]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


#### 계층 색인된 데이터는 약간 복잡하다.

In [668]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})

In [669]:
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])

In [670]:
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [671]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

#### 이런 경우에는 리스트로 여러 개의 칼럼을 지정해서 머지해야 한다(중복되는 색인 값을 다룰 때는 주의)

In [672]:
# key1, key2와 right_index가 같으니 됨
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [673]:
# key1과 right_index는 다르다. key2에 해당하는 연도가 없어서 Error가 난다.
pd.merge(lefth, righth, left_on=['key1'], right_index=True)

ValueError: len(left_on) must equal the number of levels in the index of "right"

In [675]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


#### 양쪽에 공통으로 있는 여러 개의 색인을 머지하는 것도 가능하다.

In [676]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                  columns=['Ohio', 'Nevada'])

In [677]:
right2 = DataFrame([[7., 8.,], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

In [678]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [679]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [680]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [681]:
pd.merge(left2, right2, left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


#### 색인으로 머지할 때 DataFrame의 join 메서드 편리성

- join 메서드는 **칼럼이 겹치지 않으며 완전히 같거나 유사한 색인 구조**를 가진 여러 개의 DataFrame 객체를 병합할 때 사용 가능

In [682]:
# 위와 똑같은 결과인데 join을 사용하면 훨씬 깔끔하다
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [683]:
# 색인 기준으로 머지할 때 사용!
left2.join(right2, how='inner')

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


#### 과거에 작성된 pandas의 일부 코드 제약으로 인해 DataFrame의 join 메서드는 왼쪽 우선 조인 수행

- join 메서드를 호출한 DataFrame의 칼럼 중 하나에 대해서 조인을 수행하는 것도 가능

In [684]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [685]:
right1

,group_val
a,3.5
b,7.0


In [686]:
# 왼쪽 우선 조인이라 c가 있음
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [687]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])

In [688]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [689]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [690]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [691]:
#두 개의 dataFrame을 이어붙일 수 있다. (concat메서드와 비슷함)
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [692]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [693]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [694]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [695]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 7.1.3 축 따라 이어붙이기

- concatenation(이어 붙이기)
- binding(연결)
- stacking(적층)
- NumPy는 ndarray를 연결하는 concatenate 함수 제공

In [696]:
arr = np.arange(12).reshape((3, 4))

In [697]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [698]:
type(arr)

numpy.ndarray

In [699]:
# axis = column, 1 = 세로
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [700]:
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

#### Series나 DataFrame 같은 pandas 객체의 컨텍스트 내부에는 축마다 이름이 있어서 배열을 쉽게 이어붙일 수 있도록 되어 있다. 다음 특이 사항 고려

- 만약에 연결하려는 두 객체의 색인이 서로 다르다면 결과는 그 색인의 교집합이어야 하는가 아니면 합집합이어야 하는가?
- 합쳐진 결과에서 합쳐져기 전 객체의 데이터를 구분할 수 있는가?
- 어떤 축으로 연결할 것인가?


#### 색인이 겹치지 않는 3개의 Series 객체

In [701]:
s1 = Series([0, 1], index=['a', 'b'])

In [702]:
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])

In [703]:
s3 = Series([5, 6], index=['f', 'g'])

In [704]:
s1

a    0
b    1
dtype: int64

In [705]:
s2

c    2
d    3
e    4
dtype: int64

In [706]:
s3

f    5
g    6
dtype: int64

#### 이 세 객체를 리스트로 묶어서 concat 함수에 전달하면 값과 인덱스를 연결한다.

In [707]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [708]:
# Series인 것을 확인할 수 있다.
type( pd.concat([s1, s2, s3]) )

pandas.core.series.Series

#### concat 함수는 axis = 0 기본값. 새로운 Series 객체 생성

- 만약 axis=1을 넘긴다면 결과는 Series가 아니라 DataFrame이 될 것이다.(axis=1은 칼럼을 의미)

In [709]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [710]:
# DataFrame인 것을 확인할 수 있다
type( pd.concat([s1, s2, s3], axis=1) )

pandas.core.frame.DataFrame

#### 겹치는 축이 없기 때문에 외부조인(outer 메서드)으로 정렬된 합집합을 얻었지만 join='inner'를 넘겨서 교집합을 구할 수도 있다.

In [711]:
s4 = pd.concat([s1 * 5, s3])

In [712]:
s1

a    0
b    1
dtype: int64

In [713]:
s1 * 5

a    0
b    5
dtype: int64

In [714]:
s3

f    5
g    6
dtype: int64

In [715]:
s4

a    0
b    5
f    5
g    6
dtype: int64

In [716]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [717]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


#### join_axes 인자로 머지하려는 축을 직접 지정도 가능

In [718]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


- Series를 이어붙이기 전의 개별 Series를 구분할 수 없다는 문제가 생기는데, 이 경우 이어붙인 축에 대해 계층적 색인을 생성하여 식별이 가능하도록 할 수 있다.
- **계층적 색인을 사용하려면 keys 인자 사용**

In [719]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [720]:
s1

a    0
b    1
dtype: int64

In [721]:
s2

c    2
d    3
e    4
dtype: int64

In [722]:
s3

f    5
g    6
dtype: int64

In [723]:
# s1 = one, s2 = two, s3 = three로 할당
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [724]:
# unstack 함수에 대한 자세한 내용은 나중에 알아보자
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


#### Series는 axis=1로 합병할 경우 keys는 DataFrame의 칼럼 제목이 된다.

In [725]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### DataFrame 객체에 대해서도 지금까지와 같은 방식으로 적용 가능

In [726]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], 
                columns=['one', 'two'])

In [727]:
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])

In [728]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [729]:
df2

,three,four
a,5,6
c,7,8


In [730]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

#### 리스트 대신 객체의 딕셔너리를 매개변수로 넘긴다면 딕셔너리의 키가 keys 옵션으로 사용됨

In [731]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [732]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [733]:
pd.concat?

In [734]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper'])

upper level1     level2     
         one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [735]:
# 현재 df에서 names는 2개까지만 됨. 
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower', 'test'])

ValueError: Length of names must match number of levels in MultiIndex.

#### 마지막으로, DataFrame의 로우 색인이 분석에 불필요한 경우에는 어떻게 할 것인가?
#### 그 때는 ignore_index = True 옵션 적용

In [809]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [810]:
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [811]:
df1

,a,b,c,d
0,-0.218153,0.772942,-0.728643,0.215523
1,-2.056737,-1.248733,1.266970,0.722045
2,-0.954567,0.943233,1.192702,1.035828


In [812]:
df2

,b,d,a
0,1.031435,0.179642,-0.625160
1,1.754117,0.665097,0.996054


In [813]:
# 공통된 column의 뒷부분에 이어붙으며 index가 그대로 이어짐.
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.218153,0.772942,-0.728643,0.215523
1,-2.056737,-1.248733,1.266970,0.722045
2,-0.954567,0.943233,1.192702,1.035828
3,-0.625160,1.031435,NaN,0.179642
4,0.996054,1.754117,NaN,0.665097


In [814]:
pd.concat([df1, df2])

,a,b,c,d
0,-0.218153,0.772942,-0.728643,0.215523
1,-2.056737,-1.248733,1.266970,0.722045
2,-0.954567,0.943233,1.192702,1.035828
0,-0.625160,1.031435,NaN,0.179642
1,0.996054,1.754117,NaN,0.665097


#### concat 함수 인자

인자 | 설명
--- | ---
objs | 이어붙일 pandas 객체의 사전이나 리스트. 필수 인자
axis | 이어붙일 축 방향. 기본값은 0
join | 조인 방식. 'inner'(내부조인, 교집합)와 'outer'(외부조인, 합집합)가 있으며 기본값은 'outer'
join_axes | 합집합/교집합을 수행하는 대신 다른 n-1 축으로 사용할 색인을 지정한다.
keys | 이어붙일 객체나 이어붙인 축에 대한 계층 색인을 생성하는 데 연관된 값이다. 리스트나 임의의 값이 들어있는 배열, 튜플의 배열 또는 배열의 리스트(levels 옵션에 다차원 배열이 넘어온 경우)가 될 수 있다.
levels | 계층 색인 레벨로 사용할 색인을 지정한다. keys가 넘어온 경우 여러 개의 색인을 지정한다.
names | keys나 levels 혹은 둘 다 있을 경우, 생성된 계층 레벨을 위한 이름
verify_integrity | 이어붙인 객체에 중복되는 축이 있는지 검사하고 있다면 예외를 발생시킨다. 기본값은 False로, 중복을 허용한다.
ignore_index | 이어붙인 축의 색인을 유지하지 않고 range(total_length)로 새로운 색인을 생성한다.

### 7.1.4 겹치는 데이터 합치기

- 데이터를 합칠 때 머지나 이어붙이기로는 불가능한 상황이 있는데, 두 데이터셋의 색인이 일부 겹치거나 전체가 겹치는 경우가 그렇다.
- 벡터화된 if-else 구문을 표현하는 NumPy의 Where 함수를 통해 자세히 알아보자

In [815]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])

In [816]:
b = Series(np.arange(len(a), dtype=np.float64),
            index=['f', 'e', 'd', 'c', 'b', 'a'])

In [817]:
b[-1] = np.nan

In [818]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [819]:
np.arange(len(a))

array([0, 1, 2, 3, 4, 5])

In [820]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [821]:
# pd.isnull(a)에서 null은 것은 True을 돌려준다.
# 삼항 연산자처럼 null인 True가 있으면 b값을 대입하고
# null이 False면 a값을 대입한다.
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [822]:
pd.isnull(a)

f     True
e    False
d     True
c    False
b    False
a     True
dtype: bool

#### Series 객체의 combine_first라는 메서드는 위와 동일한 연산을 제공, 데이터 정렬 기능까지 제공

In [823]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [824]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [825]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [826]:
b[:-2].combine_first

<bound method Series.combine_first of f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64>

In [827]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [828]:
b.combine_first(a)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [829]:
# a가 False일 때만 b 값을 참조한다.
# a를 먼저 combine 하되 False라면 b값 참조
a.combine_first(b)

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [830]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

#### DataFrame에서 combine_first 메서드는 칼럼에 대해 같은 동작

- 이를 통해 호출하는 객체에서 누락된 데이터를 인자로 넘긴 객체에 있는 값으로 채워 넣기 가능

In [831]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})

In [832]:
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})

In [833]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [834]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [835]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 7.2 reshaping, 피벗연산

- 재형성(reshaping, 피벗 연산): 표형식의 데이터를 재배치하는 다양한 기본 연산

### 7.2.1 계층적 색인으로 재형성하기

- stack: 데이터의 칼럼을 로우로 피벗 또는 회전시킨다.
- unstack: 로우를 칼럼으로 피벗시킨다.

In [836]:
# 문자열이 담긴 배열을 로우와 칼럼의 색인으로하는 작은 DataFrame
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))

In [837]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [838]:
# 문자열이 담긴 배열을 로우와 칼럼의 색인으로하는 작은 DataFrame
# pd.Index로 index를 설정 후에 name을 입력하기 위해서 저렇게 한 것.
# 기본적으로는 columns에 list만 넘겨도 생성 됨
data2 = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=['one', 'two', 'three'])

In [839]:
data2

,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


#### stack 메서드를 사용하면 칼럼이 로우로 피벗되어 다음과 같은 Series 객체를 반환

In [840]:
result = data.stack()

In [841]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [842]:
type(result)

pandas.core.series.Series

#### unstack 메서드를 사용하면 앞에서 얻은 계층적 색인을 가진 Series로부터 DataFrame을 얻을 수 있다.

In [843]:
# 보통 가장 안쪽에 있는 one, two, three부터 컬럼으로 얻어냄
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [844]:
type( result.unstack() )

pandas.core.frame.DataFrame

#### 보통 가장 안쪽에 있는 것부터 얻어오는데(stack도 마찬가지), 레벨 이름이나 숫자를 전달해서 얻어낼 단계를 지정할 수 있다.

- 레벨이름이 0이면 첫번쨰 index를 컬럼으로
- 레벨이름이 1이면 두번째 index를 컬럼으로
- 선택한 이름을 컬럼으로

In [845]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [846]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [847]:
result.unstack(1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [848]:
result.unstack('number')

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


- 해당 레벨이 있는 모든 값이 하위 그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터가 생길 수 있다.

In [849]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [850]:
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])

In [851]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [852]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [853]:
s2

c    4
d    5
e    6
dtype: int64

In [854]:
data3 = pd.concat([s1, s2])

In [855]:
data3

a    0
b    1
c    2
d    3
c    4
d    5
e    6
dtype: int64

In [856]:
# index로 s1을 one, s2를 two로 할당
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [857]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


#### stack 메서드는 누락된 데이터를 자동으로 걸러내기 때문에 연산을 쉽게 원상복구 가능

In [858]:
# 원상복구시 NaN값을 걸러내는 것을 확인할 수 있다.
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [859]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [860]:
#null값을 걸러내는 것을 dropna 매개변수를 통해 False로 옵션을 줄 수 있다.
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

#### DataFrame을 unstack할 때, unstack() 의 괄호 안의 레벨은 결과에서 가장 낮은 단계로 간다.

In [861]:
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))

In [862]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [863]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [864]:
# unstack 할 때 결과에서 가장 낮은 레벨. 즉, side의 하위로 갔다.
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [865]:
# 하위단계로 간 state에서 stack을 한 번 더 하였을 경우 다시 side도 number의 하위로 가는 모습이다. 
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

#### Why? 피벗을 하는가?

- [피벗테이블(Pivot Table)](http://www.soongin.com/2012/10/pivot-table.html)
- 피벗테이블은 엑셀이 제공하는 가장 강력한 데이터 분석 도구입니다. 아마 저에게 엑셀의 가장 뛰어난 기능이 뭐냐고 물으신다면... 주저없이 피벗테이블이라고 얘기할 수 있습니다. 피벗테이블은 많은 양의 데이터를 빠른 시간에 다양하게 분석할 수 있는 대화형 테이블이라고 할 수 있는데요, 데이터를 분석하는 데 있어서 이만한 기능을 제공하는 프로그램도 흔치 않은 것이 사실입니다.
- 그런데 피벗(Pivot)이라는건 뭘까요? 피벗은 단어의 의미 그대로 회전중심축을 의미하는데, 원본 데이터를 사용자가 정의한 축을 중심으로 다양하게 분석(회전)해볼 수 있다는 의미입니다. 엑셀 97에서 처음 소개된 피벗테이블은 너무 복잡해보여서 사용자들이 쉽게 접근하지 못했던 것이 사실입니다. 엑셀 2007부터는 클릭 몇번으로 피벗테이블을 만들고 분석할 수 있게 되었는데, 여러분들도 저와 함께 쉽게 이해하실 수 있을겁니다.
- [오피스 팁 엑셀의 피벗테이블을 이용하여 데이터를 분석하는 팁](http://www.itworld.co.kr/news/77360)
- [피벗 테이블 보고서](http://www.jch74.com/study/eeeee1.htm)

### 7.2. 피버팅으로 데이터 나열 방식 바꾸기

- 데이터베이스나 CSV 파일에 여러 개의 시계열 데이터를 저장하는 일반적인 방법은 시간순으로 나열하는 방법
- 컬럼 이름 변경
- 컬럼 합치기

#### names로 지정해주지 않으면 pivot 할 때 error 발생

#### pivot.csv파일을 생성하여 피벗

    date, item, value
    1959-03-31 00:00:00, realgdp, 2710.349
    1959-03-31 00:00:00, infl, 0.000
    1959-03-31 00:00:00, unemp, 5.800
    1959-06-30 00:00:00, realgdp, 2778.801
    1959-06-30 00:00:00, infl, 2.340
    1959-06-30 00:00:00, unemp, 5.100
    1959-09-30 00:00:00, realgdp, 2775.488
    1959-09-30 00:00:00, infl, 2.740
    1959-09-30 00:00:00, unemp, 5.300
    1959-12-31 00:00:00, realgdp, 2785.204        

In [866]:
%%writefile data/pivot.csv
date, item, value
1959-03-31 00:00:00, realgdp, 2710.349
1959-03-31 00:00:00, infl, 0.000
1959-03-31 00:00:00, unemp, 5.800
1959-06-30 00:00:00, realgdp, 2778.801
1959-06-30 00:00:00, infl, 2.340
1959-06-30 00:00:00, unemp, 5.100
1959-09-30 00:00:00, realgdp, 2775.488
1959-09-30 00:00:00, infl, 2.740
1959-09-30 00:00:00, unemp, 5.300
1959-12-31 00:00:00, realgdp, 2785.204        

Writing data/pivot.csv


In [867]:
# csv 파일에서 1번째 줄인 header를 지우면 어떤 컬럼인지 모르니까 차라리 skiprows=1 을 해주는게 낫겠다.
# pivot 실행 시 pivot함수는 names로 정의된 컬럼명만 인식한다.
ldata = pd.read_csv('data/pivot.csv',  names=['date', 'item', 'value'])

In [868]:
ldata[:10]

,date,item,value
0,date,item,value
1,1959-03-31 00:00:00,realgdp,2710.349
2,1959-03-31 00:00:00,infl,0.000
3,1959-03-31 00:00:00,unemp,5.800
4,1959-06-30 00:00:00,realgdp,2778.801
5,1959-06-30 00:00:00,infl,2.340
6,1959-06-30 00:00:00,unemp,5.100
7,1959-09-30 00:00:00,realgdp,2775.488
8,1959-09-30 00:00:00,infl,2.740
9,1959-09-30 00:00:00,unemp,5.300


In [869]:
type(ldata)

pandas.core.frame.DataFrame

In [870]:
# 1번째 인자: 로우 인덱스로 사용될 칼럼 이름
# 2번째 인자: 칼럼 인덱스로 사용될 칼럼 이름
# 3번째 인자: DataFrame에 채워 넣을 값을 담고 있는 칼럼
pivoted = ldata.pivot('date', 'item', 'value')

In [871]:
pivoted.head()

item,infl,item,realgdp,unemp
date,,,,
1959-03-31 00:00:00,0.000,None,2710.349,5.800
1959-06-30 00:00:00,2.340,None,2778.801,5.100
1959-09-30 00:00:00,2.740,None,2775.488,5.300
1959-12-31 00:00:00,None,None,2785.204,None
date,None,value,None,None


- 한 번에 2개의 칼럼 변형

In [872]:
ldata['value2'] = np.random.randn(len(ldata))

In [873]:
ldata[:10]

,date,item,value,value2
0,date,item,value,1.254051
1,1959-03-31 00:00:00,realgdp,2710.349,-0.071556
2,1959-03-31 00:00:00,infl,0.000,1.140204
3,1959-03-31 00:00:00,unemp,5.800,-0.139397
4,1959-06-30 00:00:00,realgdp,2778.801,0.130148
5,1959-06-30 00:00:00,infl,2.340,0.673429
6,1959-06-30 00:00:00,unemp,5.100,-0.439194
7,1959-09-30 00:00:00,realgdp,2775.488,-0.450105
8,1959-09-30 00:00:00,infl,2.740,1.543588
9,1959-09-30 00:00:00,unemp,5.300,-0.174925


In [874]:
# 3번째 인자 생략하면 계층적 인덱스로 보여줌
pivoted = ldata.pivot('date', 'item')

In [875]:
pivoted[:5]

value                                       value2  \
item                   infl    item            realgdp   unemp      infl   
date                                                                       
1959-03-31 00:00:00   0.000    None           2710.349   5.800  1.140204   
1959-06-30 00:00:00   2.340    None           2778.801   5.100  0.673429   
1959-09-30 00:00:00   2.740    None           2775.488   5.300  1.543588   
1959-12-31 00:00:00    None    None   2785.204            None       NaN   
date                   None   value               None    None       NaN   

                                                   
item                     item   realgdp     unemp  
date                                               
1959-03-31 00:00:00       NaN -0.071556 -0.139397  
1959-06-30 00:00:00       NaN  0.130148 -0.439194  
1959-09-30 00:00:00       NaN -0.450105 -0.174925  
1959-12-31 00:00:00       NaN -1.766967       NaN  
date                 1.254051       NaN       NaN

In [876]:
# 계층적 인덱스로 되어 있는 것을 ['value']로 좁혀주고 5개만 보여준다.
pivoted['value'][:5]

item,infl,item,realgdp,unemp
date,,,,
1959-03-31 00:00:00,0.000,None,2710.349,5.800
1959-06-30 00:00:00,2.340,None,2778.801,5.100
1959-09-30 00:00:00,2.740,None,2775.488,5.300
1959-12-31 00:00:00,None,None,2785.204,None
date,None,value,None,None


- pivot: set_index를 사용해서 계층적 색인을 만들고 unstack 메서드를 이용해서 형태를 변경하는 단축키같은 메서드

In [877]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [878]:
unstacked[:7]

value                                       value2  \
item                   infl    item            realgdp   unemp      infl   
date                                                                       
1959-03-31 00:00:00   0.000    None           2710.349   5.800  1.140204   
1959-06-30 00:00:00   2.340    None           2778.801   5.100  0.673429   
1959-09-30 00:00:00   2.740    None           2775.488   5.300  1.543588   
1959-12-31 00:00:00    None    None   2785.204            None       NaN   
date                   None   value               None    None       NaN   

                                                   
item                     item   realgdp     unemp  
date                                               
1959-03-31 00:00:00       NaN -0.071556 -0.139397  
1959-06-30 00:00:00       NaN  0.130148 -0.439194  
1959-09-30 00:00:00       NaN -0.450105 -0.174925  
1959-12-31 00:00:00       NaN -1.766967       NaN  
date                 1.254051       NaN       NaN

In [879]:
# 위에서 unstack을 하니 item 로우가 컬럼으로 이동했다.
unstacked = ldata.set_index(['date', 'item'])

In [880]:
# date, item이 로우
# value, value2가 열
unstacked[:7]

value    value2
date                item                         
date                 item         value  1.254051
1959-03-31 00:00:00  realgdp   2710.349 -0.071556
                     infl         0.000  1.140204
                     unemp        5.800 -0.139397
1959-06-30 00:00:00  realgdp   2778.801  0.130148
                     infl         2.340  0.673429
                     unemp        5.100 -0.439194

In [881]:
# set_index가 로우 설정
# set_index로 설정되지 않은 것들은 모두 열로 이동
unstacked = ldata.set_index(['date'])

In [882]:
unstacked

,item,value,value2
date,,,
date,item,value,1.254051
1959-03-31 00:00:00,realgdp,2710.349,-0.071556
1959-03-31 00:00:00,infl,0.000,1.140204
1959-03-31 00:00:00,unemp,5.800,-0.139397
1959-06-30 00:00:00,realgdp,2778.801,0.130148
1959-06-30 00:00:00,infl,2.340,0.673429
1959-06-30 00:00:00,unemp,5.100,-0.439194
1959-09-30 00:00:00,realgdp,2775.488,-0.450105
1959-09-30 00:00:00,infl,2.740,1.543588


## 7.3 데이터 변형

- 필터링, 정제 및 다른 변형 역시 중요한 연산

### 7.3.1 중복 제거하기

In [883]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})

In [884]:
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [885]:
#간단한 파이썬 문법. 문자열value에 수를 곱할 수가 있다.
['one'] * 3 + ['two'] * 4

['one', 'one', 'one', 'two', 'two', 'two', 'two']

In [886]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [887]:
data2 = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 3, 4]})

In [888]:
# 2개 열이 모두 같아야 중복으로 인정 됨
data2.duplicated()

0    False
1     True
2    False
3    False
4     True
5     True
6    False
dtype: bool

In [889]:
# druplicated 배열이 False인 DataFrame 반환
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [890]:
data['v1'] = range(7)

In [891]:
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [892]:
# k1컬럼의 대한 중복 value를 표시함
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [893]:
#k1컬럼에 대한 중복 value를 제거하고 중복된 값들 중 마지막 로우를 얻어옴(take_last)
data.drop_duplicates(['k1'], take_last=True)

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  from ipykernel import kernelapp as app


,k1,k2,v1
2,one,2,2
6,two,4,6


In [894]:
data.drop_duplicates(['k1'], take_last=False)

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: the take_last=False keyword is deprecated, use keep='first' instead
  if __name__ == '__main__':


,k1,k2,v1
0,one,1,0
3,two,3,3


In [895]:
data.drop_duplicates(['k1', 'k2'])

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


In [896]:
# take_last=False는 5를 선택
# take_last=True는 6을 선택
data.drop_duplicates(['k1', 'k2'], take_last=True)

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  app.launch_new_instance()


,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


In [897]:
data.drop_duplicates?

    Type:        instancemethod
    String form:
    <bound method DataFrame.drop_duplicates of     k1  k2  v1
    0  one   1   0
    1  one   1   1
    2  one   2   2
    3  two   3   3
    4  two   3   4
    5  two   4   5
    6  two   4   6>
    File:        /Library/Python/2.7/site-packages/pandas-0.12.0_307_g3a2fe0b-py2.7-macosx-10.8-intel.egg/pandas/core/frame.py
    Definition:  data.drop_duplicates(self, cols=None, take_last=False, inplace=False)
    Docstring:
    Return DataFrame with duplicate rows removed, optionally only
    considering certain columns

    Parameters
    ----------
    cols : column label or sequence of labels, optional
        Only consider certain columns for identifying duplicates, by
        default use all of the columns
    take_last : boolean, default False
        Take the last observed row in a row. Defaults to the first row
    inplace : boolean, default False
        Whether to drop duplicates in place or to return a copy

    Returns
    -------
    deduplicated : DataFrame

### 7.3.2 함수나 매핑 이용해 데이터 변형하기

- DataFrame의 칼럼이나 Series, 배열 안의 값을 기반으로 데이터의 형태를 변형하고 싶을 때

In [898]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami',
                           'honey ham', 'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [899]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


- 해당 육류가 어떤 동물의 고기인지 알려줄 수 있는 칼럼을 하나 추가한다고 가정
- 육류별 동물을 담고 있는 사전 데이터 작성

In [900]:
meat_to_animal = {
                  'bacon': 'pig',
                  'pulled pork': 'pig',
                  'pastrami': 'cow',
                  'corned beef': 'cow',
                  'honey ham': 'pig',
                  'nova lox': 'salmon'
                  }

- Series의 map 메서드는 사전류의 객체나 어떤 함수를 받을 수 있는데, 이 데이터에는 육류의 이름에 대∙소문자가 섞여 있는 사소한 문제가 있으므로 모두 소문자로 변경

#### 3단계로 풀어 헤쳐서 함수의 역할 살펴보기

- 처음에는 한 번에 읽을 수 없으니 조각조각 코드를 나눠서 실행해 본다.
- 하나씩 하나씩 퍼즐 맞추듯이 조립하여 의미를 확인해 본다.

In [901]:
# 먼저 data['food']에 어떤 데이터가 있는지 확인
data['food']

0          bacon
1    pulled pork
2          bacon
3       Pastrami
4    corned beef
5          Bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [902]:
data['food'].map?

In [ ]:
data['food'].map

In [903]:
data['food'].map

<bound method Series.map of 0          bacon
1    pulled pork
2          bacon
3       Pastrami
4    corned beef
5          Bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object>

    map 1번째 인자로 함수를 넘기고, 2번째 인자로 연속된 형태의(리스트, 튜플 같은) 자료형을 넘기면 된다.

    Type:        builtin_function_or_method
    String form: <built-in function map>
    Namespace:   Python builtin
    Docstring:
    map(function, sequence[, sequence, ...]) -> list

    Return a list of the results of applying the function to the items of
    the argument sequence(s).  If more than one sequence is given, the
    function is called with an argument list consisting of the corresponding
    item of each sequence, substituting None for missing values when not all
    sequences have the same length.  If the function is None, return a list of
    the items of the sequence (or a list of tuples if more than one sequence).

In [904]:
data['food'].map

<bound method Series.map of 0          bacon
1    pulled pork
2          bacon
3       Pastrami
4    corned beef
5          Bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object>

In [905]:
# data['food']에 map 함수를 적용하는데 소문자로 모두 변경하는 함수 적용
data['food'].map(str.lower)

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [906]:
# 또 map으로 meat_to_animal dictionary를 넘긴다.
# bacon -> pig로, pastrami -> cow로 변경
data['food'].map(str.lower).map(meat_to_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [907]:
# 변경된 것들을 animal 열을 새로 생성하고 여기에 대입
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)

In [908]:
# food열의 대∙소문자는 안 바뀐 것을 알 수 있다.
# 대∙소문자 비교한건 animal 필드에 적용하기 위한 것인 것임을 확인할 수 있다.
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


- 물론 함수를 넘겨서 같은 일을 수행할 수 있다.

In [909]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

- map 메서드를 사용하면 데이터의 요소별 변형 및 데이터를 다듬는 작업 편리

### 7.3.3 값 치환하기

- fillna 메서드: 누락된 값을 채우는 일은 일반적인 값 치환 작업
- 위에서 살펴봤듯이 map 메서드를 한 객체 안에서 값의 부분집합을 변경하는 데 사용했다면,
- replace 메서드는 같은 작업에 대해서 좀 더 간단하고 유연한 방법 제공

In [910]:
data = Series([1., -999., 2., -999., -1000., 3.])

In [911]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

- -999는 누락된 데이터를 나타내기 위한 값
- 이 값은 replace 메서드를 이용해서 pandas에서 인식할 수 있는 NA값으로 치환된 새로운 Series를 생성

In [912]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

- 당연히 여러 개의 값도 한 번에 치환 가능

In [913]:
# 1개 이상을 변경하려면 list로 넘기면 모두 알아서 변경해준다.
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [914]:
# 변경하고 싶은 문자열도 list로 넘기면 순서에 맞게 변경해 줌
data.replace([-999, -1000], [999, 1000])

0       1.0
1     999.0
2       2.0
3     999.0
4    1000.0
5       3.0
dtype: float64

In [915]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [916]:
# 사전으로 넘겨도 된다.
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.3.4 축 색인 이름 바꾸기

- Series의 값처럼 축 이름 역시 유사한 방식으로 함수나 새롭게 바꿀 값으로 이용해서 변형
- 새로운 자료 구조를 만들지 않고 그 자리에서 바로 축 이름을 변경하는 것이 가능

In [917]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index = ['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])

In [918]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


#### map 함수 사용법
* 함수의 인자로 함수를 넘기면 된다.

In [919]:
data.index.map(str.upper)

array(['OHIO', 'COLORADO', 'NEW YORK'], dtype=object)

In [920]:
data.index = data.index.map(str.upper)

In [921]:
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


- rename 메서드: 원래 객체를 변경하지 않고 새로운 객체를 생성

In [922]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [923]:
# 위에서 index를 대문자로 바꿔줬다.
# rename을 사용해서 원본은 바뀌지 않았다.
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


- rename 메서드: 사전 형식의 객체를 이용해서 축 이름 중 일부만 변경하는 것도 가능

In [924]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [925]:
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


- rename 메서드: DataFrame을 직접 복사해서 index와 columns 속성을 갱신할 필요없이 바로 변경도 가능
- 원본 데이터를 바로 변경하려면 inplace = True 옵션 설정
- 원본 데이터를 바로 변경하기 때문에 매우 주의해서 사용하여야 한다.

In [926]:
# 항상 DataFrame의 참조를 반환한다.
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [927]:
# OHIO -> INDIANA로 바뀌었다.
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### 7.3.5 개별화와 양자화

- 연속성 데이터는 종종 개별로 분할하거나 분석을 위해 그룹별로 나눔
- 수업에 참여하는 학생 그룹 데이터가 있고, 나이대에 따라 분류한다고 가정

In [928]:
ages = [20, 22, 25, 27, 21, 23, 27, 31, 61, 45, 41, 32]

- 이 데이터를 pandas의 cut 함수를 사용해서 18-25, 26-35, 35-60, 60이상인 그룹으로 나눔

In [929]:
bins = [18, 25, 35, 60, 100]

In [930]:
# 1번째 인자: 나눌 list
# 2번째 인자: 나눌 기준
cats = pd.cut(ages, bins)

In [931]:
pd.cut?

    Type:        function
    String form: <function cut at 0x109e2acf8>
    File:        /Library/Python/2.7/site-packages/pandas-0.12.0_307_g3a2fe0b-py2.7-macosx-10.8-intel.egg/pandas/tools/tile.py
    Definition:  pd.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)
    Docstring:
    Return indices of half-open bins to which each value of `x` belongs.

    Parameters
    ----------
    x : array-like
        Input array to be binned. It has to be 1-dimensional.
    bins : int or sequence of scalars
        If `bins` is an int, it defines the number of equal-width bins in the
        range of `x`. However, in this case, the range of `x` is extended
        by .1% on each side to include the min or max values of `x`. If
        `bins` is a sequence it defines the bin edges allowing for
        non-uniform bin width. No extension of the range of `x` is done in
        this case.
    right : bool, optional
        Indicates whether the bins include the rightmost edge or not. If
        right == True (the default), then the bins [1,2,3,4] indicate
        (1,2], (2,3], (3,4].
    labels : array or boolean, default None
        Labels to use for bin edges, or False to return integer bin labels
    retbins : bool, optional
        Whether to return the bins or not. Can be useful if bins is given
        as a scalar.

    Returns
    -------
    out : Categorical or array of integers if labels is False
    bins : ndarray of floats
        Returned only if `retbins` is True.

    Notes
    -----
    The `cut` function can be useful for going from a continuous variable to
    a categorical variable. For example, `cut` could convert ages to groups
    of age ranges.

    Any NA values will be NA in the result.  Out of bounds values will be NA in
    the resulting Categorical object


    Examples
    --------
    >>> cut(np.array([.2, 1.4, 2.5, 6.2, 9.7, 2.1]), 3, retbins=True)
    (array([(0.191, 3.367], (0.191, 3.367], (0.191, 3.367], (3.367, 6.533],
           (6.533, 9.7], (0.191, 3.367]], dtype=object),
     array([ 0.1905    ,  3.36666667,  6.53333333,  9.7       ]))
    >>> cut(np.ones(5), 4, labels=False)
    array([2, 2, 2, 2, 2])

In [932]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

- pandas에서 반환하는 객체는 Categorical 객체라는 특수한 객체인데, 이 객체는 그룹 이름이 담긴 배열

In [933]:
cats.labels

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  if __name__ == '__main__':


array([0, 0, 0, 1, 0, 0, 1, 1, 3, 2, 2, 1], dtype=int8)

In [934]:
#각 범위에 속해있는 value의 개수를 카운트
pd.value_counts(cats)

(18, 25]     5
(25, 35]     4
(35, 60]     2
(60, 100]    1
dtype: int64

- 간격을 나타내는 표기법은 (로 시작해서 ]로 끝난다.
- (: 괄호쪽의 값은 포함하지 않고 
- [: 대괄호 쪽의 값은 포함하는 간격을 나타냄
- right=False를 넘겨서 괄호와 대괄호의 위치 변경 가능

In [935]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, object): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [936]:
bins

[18, 25, 35, 60, 100]

- labels 옵션으로 그룹의 이름을 직접 넘겨줄 수도 있다.

In [937]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [938]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

- cut 함수에 명시적으로 그룹의 경계 값을 넘기지 않고 그룹의 개수를 넘겨주면 데이터 내에서 최소값과 최대값을 기준으로 균등한 길이의 그룹을 자동으로 계산

In [939]:
data = np.random.randn(20)

In [940]:
data

array([ 1.39519104, -0.71285648,  0.62095485, -0.49055134, -1.41646533,
        0.23258803,  2.16236988, -0.70373559, -0.70372827,  0.81163897,
        0.68424482, -1.3384309 ,  1.88070991,  1.32814127, -0.34869066,
       -0.55312234,  0.50238564,  1.54416756, -0.57426428,  0.89054548])

In [941]:
# data, 그룹의 갯수, 소수점 자리 표현
pd.cut(data, 4, precision=2)

[(1.27, 2.16], (-1.42, -0.52], (0.37, 1.27], (-0.52, 0.37], (-1.42, -0.52], ..., (-1.42, -0.52], (0.37, 1.27], (1.27, 2.16], (-1.42, -0.52], (0.37, 1.27]]
Length: 20
Categories (4, object): [(-1.42, -0.52] < (-0.52, 0.37] < (0.37, 1.27] < (1.27, 2.16]]

- qcut: 표본 변위치를 기반으로 데이터를 나눠준다. 데이터의 분산에 따라 각각의 그룹마다 데이터의 개수가 다르게 나누어지는 경우가 많다.
- qcut: 표준 변위치를 사용하기 때문에 적당히 같은 크기의 그룹으로 나눌 수 있다.

In [942]:
data = np.random.randn(1000) # 정규분포

In [943]:
cats = pd.qcut(data, 4) # 4개 그룹으로 균등하게 나눔

In [944]:
cats

[[-3.646, -0.7], (0.64, 3.526], (-0.0398, 0.64], (-0.7, -0.0398], (-0.7, -0.0398], ..., (-0.0398, 0.64], (-0.7, -0.0398], [-3.646, -0.7], [-3.646, -0.7], [-3.646, -0.7]]
Length: 1000
Categories (4, object): [[-3.646, -0.7] < (-0.7, -0.0398] < (-0.0398, 0.64] < (0.64, 3.526]]

In [945]:
pd.value_counts(cats)

(0.64, 3.526]      250
(-0.0398, 0.64]    250
(-0.7, -0.0398]    250
[-3.646, -0.7]     250
dtype: int64

In [946]:
data2 = pd.cut(data, 4, precision=2)

In [947]:
# qcut과 비교를 위해
pd.value_counts(data2)

(-0.06, 1.73]     475
(-1.85, -0.06]    459
(-3.65, -1.85]     35
(1.73, 3.53]       31
dtype: int64

- cut함수와 유사하게 변위치를 직접 지정해줄 수 있다(변위치 값은 0부터 1까지)

In [948]:
cats2 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

In [949]:
cats2

[(-1.278, -0.0398], (-0.0398, 1.222], (-0.0398, 1.222], (-1.278, -0.0398], (-1.278, -0.0398], ..., (-0.0398, 1.222], (-1.278, -0.0398], (-1.278, -0.0398], (-1.278, -0.0398], [-3.646, -1.278]]
Length: 1000
Categories (4, object): [[-3.646, -1.278] < (-1.278, -0.0398] < (-0.0398, 1.222] < (1.222, 3.526]]

In [950]:
pd.value_counts(cats2)

(-0.0398, 1.222]     400
(-1.278, -0.0398]    400
(1.222, 3.526]       100
[-3.646, -1.278]     100
dtype: int64

- 그룹 분석과 변위치를 다룰 때는 cut과 qcut 함수 같은 이산 함수가 특히 더 유용
- 이 내용은 수집과 그룹 연산에 대한 장에서 다시 한 번 살펴봄

### 7.3.6 특이값 찾아내고 제외하기

- 배열연산을 수행할 때는 특이값(outlier)을 제외하거나 적당한 값으로 대체하는 것 중요
- 실제 실무에서도 outlier는 제거한다. 이 하나의 특이값이 나머지 값들을 변화시킬 수도 있기 때문이다.

In [951]:
np.random.seed?

In [952]:
np.random.seed(12345)

In [953]:
data = DataFrame(np.random.randn(1000, 4))

In [954]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


- 한 칼럼에서 절대 값이 3 초과하는 값 찾아내기

In [955]:
col = data[3]

In [956]:
col[np.abs(col) > 3] #절대값을 구하는 함수 abs()

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [957]:
data[np.abs(data) > 3]

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,3.248944,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [958]:
# 절대값 3을 초과하는 값이 들어있는 모든 로우를 선택하려면 any 메서드 사용
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


- 유니버설 함수인 np.sign은 주어진 값이 부호에 따라 1이나 -1이 담긴 배열을 반환

In [959]:
# 이렇게하면 sign값만 추출할 수 있다.
np.sign(data[2])

0     -1.0
1      1.0
2      1.0
3      1.0
4      1.0
5      1.0
6      1.0
7     -1.0
8     -1.0
9     -1.0
10    -1.0
11     1.0
12     1.0
13     1.0
14     1.0
15     1.0
16    -1.0
17    -1.0
18     1.0
19    -1.0
20     1.0
21    -1.0
22     1.0
23     1.0
24     1.0
25    -1.0
26     1.0
27    -1.0
28    -1.0
29     1.0
      ... 
970    1.0
971   -1.0
972   -1.0
973    1.0
974    1.0
975    1.0
976    1.0
977    1.0
978    1.0
979   -1.0
980   -1.0
981    1.0
982    1.0
983    1.0
984    1.0
985    1.0
986    1.0
987    1.0
988   -1.0
989    1.0
990   -1.0
991    1.0
992   -1.0
993   -1.0
994    1.0
995   -1.0
996   -1.0
997    1.0
998   -1.0
999    1.0
Name: 2, dtype: float64

In [960]:
# 절대값 3을 넘는 것들을 수정하는데 원래 데이터의 sign값을 유지하면서 * 3을 한다.
data[np.abs(data) > 3] = np.sign(data) * 3

In [961]:
data[(np.abs(data) >= 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.000000,-1.021228
97,-0.774363,0.552936,0.106061,3.000000
102,-0.655054,-0.565230,3.000000,0.959533
305,-2.315555,0.457246,-0.025907,-3.000000
324,0.050188,1.951312,3.000000,0.963301
400,0.146326,0.508391,-0.196713,-3.000000
499,-0.293333,-0.242459,-3.000000,1.918403
523,-3.000000,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.000000,1.369891
808,-0.362528,-3.000000,1.553205,-2.186301


#### np.any 실험

In [962]:
(np.abs(data) > 3).any?

In [ ]:
(np.abs(data) > 3).any

In [963]:
(np.abs(data) > 3).any

<bound method DataFrame.any of          0      1      2      3
0    False  False  False  False
1    False  False  False  False
2    False  False  False  False
3    False  False  False  False
4    False  False  False  False
5    False  False  False  False
6    False  False  False  False
7    False  False  False  False
8    False  False  False  False
9    False  False  False  False
10   False  False  False  False
11   False  False  False  False
12   False  False  False  False
13   False  False  False  False
14   False  False  False  False
15   False  False  False  False
16   False  False  False  False
17   False  False  False  False
18   False  False  False  False
19   False  False  False  False
20   False  False  False  False
21   False  False  False  False
22   False  False  False  False
23   False  False  False  False
24   False  False  False  False
25   False  False  False  False
26   False  False  False  False
27   False  False  False  False
28   False  False  False  False
29   Fals

In [964]:
(np.abs(data) > 3).any

<bound method DataFrame.any of          0      1      2      3
0    False  False  False  False
1    False  False  False  False
2    False  False  False  False
3    False  False  False  False
4    False  False  False  False
5    False  False  False  False
6    False  False  False  False
7    False  False  False  False
8    False  False  False  False
9    False  False  False  False
10   False  False  False  False
11   False  False  False  False
12   False  False  False  False
13   False  False  False  False
14   False  False  False  False
15   False  False  False  False
16   False  False  False  False
17   False  False  False  False
18   False  False  False  False
19   False  False  False  False
20   False  False  False  False
21   False  False  False  False
22   False  False  False  False
23   False  False  False  False
24   False  False  False  False
25   False  False  False  False
26   False  False  False  False
27   False  False  False  False
28   False  False  False  False
29   Fals

In [965]:
(np.abs(data) > 3).any

<bound method DataFrame.any of          0      1      2      3
0    False  False  False  False
1    False  False  False  False
2    False  False  False  False
3    False  False  False  False
4    False  False  False  False
5    False  False  False  False
6    False  False  False  False
7    False  False  False  False
8    False  False  False  False
9    False  False  False  False
10   False  False  False  False
11   False  False  False  False
12   False  False  False  False
13   False  False  False  False
14   False  False  False  False
15   False  False  False  False
16   False  False  False  False
17   False  False  False  False
18   False  False  False  False
19   False  False  False  False
20   False  False  False  False
21   False  False  False  False
22   False  False  False  False
23   False  False  False  False
24   False  False  False  False
25   False  False  False  False
26   False  False  False  False
27   False  False  False  False
28   False  False  False  False
29   Fals

In [966]:
(np.abs(data) > 3).any

<bound method DataFrame.any of          0      1      2      3
0    False  False  False  False
1    False  False  False  False
2    False  False  False  False
3    False  False  False  False
4    False  False  False  False
5    False  False  False  False
6    False  False  False  False
7    False  False  False  False
8    False  False  False  False
9    False  False  False  False
10   False  False  False  False
11   False  False  False  False
12   False  False  False  False
13   False  False  False  False
14   False  False  False  False
15   False  False  False  False
16   False  False  False  False
17   False  False  False  False
18   False  False  False  False
19   False  False  False  False
20   False  False  False  False
21   False  False  False  False
22   False  False  False  False
23   False  False  False  False
24   False  False  False  False
25   False  False  False  False
26   False  False  False  False
27   False  False  False  False
28   False  False  False  False
29   Fals

    Type:        function
    String form: <function any at 0x102729668>
    File:        /Library/Python/2.7/site-packages/numpy-1.9.0.dev_c50e60d-py2.7-macosx-10.8-x86_64.egg/numpy/core/fromnumeric.py
    Definition:  any(a, axis=None, out=None, keepdims=False)
    Docstring:
    Test whether any array element along a given axis evaluates to True.

    Returns single boolean unless `axis` is not ``None``

    Parameters
    ----------
    a : array_like
        Input array or object that can be converted to an array.
    axis : None or int or tuple of ints, optional
        Axis or axes along which a logical OR reduction is performed.
        The default (`axis` = `None`) is perform a logical OR over all
        the dimensions of the input array. `axis` may be negative, in
        which case it counts from the last to the first axis.

        .. versionadded:: 1.7.0

        If this is a tuple of ints, a reduction is performed on multiple
        axes, instead of a single axis or all the axes as before.
    out : ndarray, optional
        Alternate output array in which to place the result.  It must have
        the same shape as the expected output and its type is preserved
        (e.g., if it is of type float, then it will remain so, returning
        1.0 for True and 0.0 for False, regardless of the type of `a`).
        See `doc.ufuncs` (Section "Output arguments") for details.
    keepdims : bool, optional
        If this is set to True, the axes which are reduced are left
        in the result as dimensions with size one. With this option,
        the result will broadcast correctly against the original `arr`.

    Returns
    -------
    any : bool or ndarray
        A new boolean or `ndarray` is returned unless `out` is specified,
        in which case a reference to `out` is returned.

    See Also
    --------
    ndarray.any : equivalent method

    all : Test whether all elements along a given axis evaluate to True.

    Notes
    -----
    Not a Number (NaN), positive infinity and negative infinity evaluate
    to `True` because these are not equal to zero.

    Examples
    --------
    >>> np.any([[True, False], [True, True]])
    True

    >>> np.any([[True, False], [False, False]], axis=0)
    array([ True, False], dtype=bool)

    >>> np.any([-1, 0, 5])
    True

    >>> np.any(np.nan)
    True

    >>> o=np.array([False])
    >>> z=np.any([-1, 4, 5], out=o)
    >>> z, o
    (array([ True], dtype=bool), array([ True], dtype=bool))
    >>> # Check now that z is a reference to o
    >>> z is o
    True
    >>> id(z), id(o) # identity of z and o              # doctest: +SKIP
    (191614240, 191614240)

### 7.3.7 치환과 임의 샘플링

- numpy.random.permutation 함수를 이용하면 **로우**를 쉽게 임의의 순서대로 재배치
- 순서를 바꾸고 싶은 만큼의 길이를 permutation 함수에 넘기면 바뀐 순서가 담긴 정수 배열 생성성

In [967]:
df = DataFrame(np.arange(5 * 4).reshape(5, 4))

In [968]:
sampler = np.random.permutation(5)

In [969]:
sampler

array([1, 0, 2, 3, 4])

In [970]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [971]:
# 행을 선택할 때 take를 사용한다
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


- 치환없이 일부만 임의로 선택하려면 permutation 함수에서 반환된 배열에서 원하는 크기 k만큼만 처음부터 잘라내면 된다.

In [972]:
# 1. df의 크기를 넘겨주어 치환을 한다.
# 2. df.take 함수로 df에 함수 적용
# 3. 이 데이터 길이는 5개이지만 0,1,2만 보고 싶을 때 [:3]으로 보여주는 것 제한
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


In [973]:
df.take(np.random.permutation(len(df)))

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19


In [974]:
# 할 때마다 바뀜
# 1. df의 크기를 넘겨주어 치환을 한다.
np.random.permutation(len(df))

array([1, 0, 4, 3, 2])

In [975]:
np.random.permutation(len(df))

array([1, 4, 0, 2, 3])

In [976]:
np.random.permutation(5)

array([0, 3, 2, 4, 1])

- 치환을 통해 표본을 생성하기 위한 가장 빠른 방법은 np.random.randint를 사용해서 생성한 난수를 이용하는 방법

In [977]:
bag = np.array([5, 7, -1, 6, 4])

In [978]:
# 1. sampler에 의해서 0, bag의 크기범위의 랜덤 integer가 10개 생성
sampler = np.random.randint(0, len(bag), size=10)

In [979]:
sampler

array([3, 0, 1, 2, 2, 3, 2, 1, 2, 0])

In [980]:
draws = bag.take(sampler)

In [981]:
# sampler 순서에 따라 bag에 있는 원소들을 배열
# sampler[0]: 1이니 bag[1]의 7
# sampler[1]: 3이니 bag[3]의 6
draws

array([ 6,  5,  7, -1, -1,  6, -1,  7, -1,  5])

### 7.3.8 표시자/더미 변수

- 통계 모델이나 기계 학습 애플리케이션을 위한 또 다른 데이터 변형은 분류 값을 더미나 표시 행렬로 변환하는 것
- 만약 어떤 DataFrame의 한 칼럼에 k가지의 값이 있다면 k개의 칼럼이 있는 DataFrame이나 행렬을 만들고 값으로는 1과 0을 채워 넣을 것이다.
- 이때 사용하는 함수가 pandas의 get_dummies 함수.

In [982]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})

In [983]:
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [984]:
# df['key']대로 key가 b이면 b컬럼에 1 세팅. 나머지는 0
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [985]:
df['key']

0    b
1    b
2    a
3    c
4    a
5    b
Name: key, dtype: object

- 표시용 DataFrame 안에 있는 칼럼에 prefix를 추가한 후 다른 데이터와 병합하고 싶을 때가 있는데, 이런 경우 get_dummies 함수의 prefix 인자를 사용

In [986]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [987]:
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [988]:
df_with_dummy = df[['data1']].join(dummies)

In [989]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [990]:
#개봉년도, 영화제목, 장르의 정보를 가진 20세기 영화 데이터
# 세개의 컬럼만 읽는다
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('data/movies.dat', sep='::',
                       header=None, names=mnames)

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


FileNotFoundError: [Errno 2] No such file or directory: 'data/movies.dat'

In [991]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [992]:
# 이걸 Unique 하게 만들어야 한다. (set.union 사용)
movies.genres

0          Animation|Children's|Comedy
1         Adventure|Children's|Fantasy
2                       Comedy|Romance
3                         Comedy|Drama
4                               Comedy
5                Action|Crime|Thriller
6                       Comedy|Romance
7                 Adventure|Children's
8                               Action
9            Action|Adventure|Thriller
10                Comedy|Drama|Romance
11                       Comedy|Horror
12                Animation|Children's
13                               Drama
14            Action|Adventure|Romance
15                      Drama|Thriller
16                       Drama|Romance
17                            Thriller
18                              Comedy
19                              Action
20                 Action|Comedy|Drama
21                Crime|Drama|Thriller
22                            Thriller
23                        Drama|Sci-Fi
24                       Drama|Romance
25                       

In [993]:
# 1. movies.genres에서 x를 하나씩 추출
# 2. x.split('|')로 나눈다. (1, 2, 3으로 나누어짐)
# 3. 여기에서 다시 중복이 없게 하기 위해 set 함수를 한 번 적용
# 4. 최종적으로 장르 generator가 생성됨
genre_iter = (set(x.split('|')) for x in movies.genres)

In [994]:
genre_iter

<generator object <genexpr> at 0x0000016FABE3E5C8>

In [995]:
set.union?

    Type:        method_descriptor
    String form: <method 'union' of 'set' objects>
    Namespace:   Python builtin
    Docstring:
    Return the union of sets as a new set.

    (i.e. all elements that are in either set.)

In [996]:
# genre_iter가 generator이기 때문에 앞에 *를 붙여주어 여러개를 받을 수 있게 해주었다.
# 어쨋든 set.union으로 generator를 1개씩 넘겨서 모두 합치고 sorted 함수로 sorting
genres = sorted(set.union(*genre_iter))

In [997]:
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

- 표시용 DataFrame을 생성하기 위해 0으로 초기화된 DataFrame을 생성

In [998]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)

In [999]:
dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1000]:
dummies[:10]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 각 영화를 순회하면서 dummies 칼럼의 각 항목을 1로 설정

In [1001]:
#해당 장르가 있는 부분만을 1로 표시
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1

In [1002]:
dummies[:10]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


- 앞에서 한 대로 movies DataFrame과 조합하면 됨

In [1003]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [1004]:
movies_windic.ix[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

In [1005]:
movies_windic.ix[1]

movie_id                                        2
title                              Jumanji (1995)
genres               Adventure|Children's|Fantasy
Genre_Action                                    0
Genre_Adventure                                 1
Genre_Animation                                 0
Genre_Children's                                1
Genre_Comedy                                    0
Genre_Crime                                     0
Genre_Documentary                               0
Genre_Drama                                     0
Genre_Fantasy                                   1
Genre_Film-Noir                                 0
Genre_Horror                                    0
Genre_Musical                                   0
Genre_Mystery                                   0
Genre_Romance                                   0
Genre_Sci-Fi                                    0
Genre_Thriller                                  0
Genre_War                                       0


- 데이터가 이보다 더 크다면 다중 멤버십을 갖는 표시용 변수를 이 방법으로 생성하는 것은 그다지 빠른 방법은 아니다. 빠르게 처리하고 싶다면 DataFrame의 내부를 조작하는 저수준의 함수를 작성해서 사용해야 한다.


- get_dummies나 cut 같은 이산 함수를 잘 조합하면 통계 애플리케이션에서 유용하게 사용가능하다.

In [1006]:
values = np.random.rand(10)

In [1007]:
values

array([ 0.90953758,  0.38441329,  0.08127486,  0.95663486,  0.04387138,
        0.14476865,  0.11766477,  0.13018305,  0.62275769,  0.08497892])

In [1008]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [1009]:
#데이터가 어느 범주에 속하는지 간단히 찾을 수 있다.
pd.get_dummies(pd.cut(values, bins))

,"(0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,0,0,0,1
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,0,0,0,1,0
9,1,0,0,0,0


In [1010]:
pd.cut(values, bins)

[(0.8, 1], (0.2, 0.4], (0, 0.2], (0.8, 1], (0, 0.2], (0, 0.2], (0, 0.2], (0, 0.2], (0.6, 0.8], (0, 0.2]]
Categories (5, object): [(0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1]]

## 7.4 문자열 다루기

- 파이썬 대중적 인기 원인: 문자열이나 텍스트 처리의 용이함이 한몫
- 텍스트 연산은 대부분 문자열 객체의 내장 메서드를 통해 간단하게 처리
- 좀 더 복잡한 패턴 매칭이나 텍스트 조작은 정규표현식 필요
- pandas는 배열 데이터 전체에 쉽게 정규표현식을 적용하고 추가적으로 누락된 데이터를 편리하게 처리할 수 있는 기능 포함

### 7.4.1 문자열 객체 메서드

- 내장 문자열 메서드만으로도 웬만한 애플리케이션들은 해결 가능

In [1011]:
val  = 'a,b, guido'

In [1012]:
# 쉼표로 구분된 문자열 분리
val.split(',')

['a', 'b', ' guido']

- strip 메서드: 공백문자(줄바꿈문자 포함) 제거

In [1013]:
pieces = [x.strip() for x in val.split(',')]

In [1014]:
pieces

['a', 'b', 'guido']

- 더하기 연산을 통해 '::' 문자열과 합칠 수 있다.

In [1015]:
first, second, third = pieces

In [1016]:
first

'a'

In [1017]:
second

'b'

In [1018]:
third

'guido'

In [1019]:
first + '::' + second + '::' + third

'a::b::guido'

- join 메서드를 많이 사용한다. (더 효율적임)

In [1020]:
# '합칠문자열'.join(합칠리스트)
'::'.join(pieces)

'a::b::guido'

- 일치하는 부분 문자열의 위치를 찾는 방법도 존재
- index나 find를 사용하는 것도 가능하지만 파이썬의 in 예약어를 사용하면 일치하는 부분 문자열을 쉽게 찾을 수 있다.

In [1021]:
'guido' in val

True

In [1022]:
val.index(',')

1

In [1023]:
val.find(':')

-1

In [1024]:
val

'a,b, guido'

#### find, index의 차이점

- index의 경우 문자열을 찾지 못하면 예외를 발생
- find의 경우 -1을 반환

In [1025]:
#없는 단어이기 때문
val.index(':')

ValueError: substring not found

In [1026]:
# count는 특정 부분 문자열이 몇 건 발견되었는지 반환
val.count(',')

2

- replace는 찾아낸 패턴을 다른 문자열로 치환
- 대체할 문자열로 비어있는 문자열을 넘겨서 패턴을 삭제하기 위한 방법으로 자주 사용

In [1027]:
val.replace(',', '::')

'a::b:: guido'

In [1028]:
val.replace(',', '')

'ab guido'

#### 파이썬 내장 문자열 메서드

인자 | 설명
--- | ---
count | 문자열에서 겹치지 않는 부분 문자열의 개수를 반환
endswith, startswith | 문자열이 주어진 접미사(접두사)로 끝날 경우 True를 반환
join | 문자열을 구분자로 하여 다른 문자열을 순서대로 이어붙인다.
index | 부분 문자열의 첫 번째 글자의 위치를 반환한다. 부분 문자열이 없다면 ValueError 예외가 발생한다.
find | 첫 번째 부분 문자열의 첫 번째 글자의 위치를 반환한다. index와 유사하지만 부분 문자열이 없을 경우 -1을 반환한다.
rfind | 마지막 부분 문자열의 첫 번째 글자의 위치를 반환한다. 부분 문자열이 없다면 -1을 반환한다.
replace | 문자열을 다른 문자열로 치환한다.
strip, rstrip, lstrip | 개행문자를 포함한 공백문자를 제거한다. lstrip은 문자열의 시작 부분에 있는 공백 문자만을 제거하고, rstrip은 문자열의 마지막 부분에 잇는 공백문자만을 제거한다.
split | 문자열을 구분자를 기준으로 부분 문자열의 리스트로 분리한다.
lower, upper | 각각 알파벳 문자를 소문자 혹은 대문자로 변환한다.
ljust, rjust | 문자열을 오른쪽 혹은 왼쪽으로 정렬하고 주어진 길이에서 문자열의 길이를 제외한 곳은 공백문자를 채워 넣어 주어진 길이를 가지는 문자열을 반환한다.

### 7.4.2 정규표현식

- 텍스트에서 문자열 패턴을 찾는 유연한 방법
- regex라 불리는 단일 표현식은 정규표현언어로 구성된 문자열
- 정규표현식을 작성하는 방법은 **그 자체로 하나의 독립된 장**으로 구성할 수 있는 만큼, 이 책에서 다루는 범위를 벗어난다. 인터넷에는 제드쇼의 '정규표현식 어렵게 배우기(Learn Regex The Hard Way) 같은 훌륭한 튜토리얼과 레퍼런스가 있다.
- re 모듈 함수는 패턴 매칭, 치환, 분리의 3가지
- 3가지는 서로 연관되어 있으며, 정규표현식은 텍스트 안에 존재하는 패턴을 표현하고 이를 여러가지 다양한 목적으로 사용할 수 있도록 되어 있다.
- 여러가지 공백문자(탭, 스페이스, 개행문자)가 포함된 문자열을 나누고 싶다면 하나 이상의 공백문자를 의미하는 \s+를 사용해서 문자열을 분리할 수 있다.

#### 정규표현식 도서

- [Yes 24 정규표현식 도서 검색](http://www.yes24.com/SearchCorner/Search?mstr_query=&query=%c1%a4%b1%d4+%c7%a5%c7%f6%bd%c4&disp_no=&title_yn=y&author_yn=y&company_yn=y&domain=all)
- [정규 표현식 완전 해부와 실습(개정판)](http://www.hanbit.co.kr/book/look.html?isbn=89-7914-226-9)
- [정규 표현식 완전 해부와 실습 리뷰](http://www.benjaminlog.com/80)

In [1029]:
# 정규표현식, 텍스트에서 문자열을 찾는 도구
# regex
# 패턴 매칭, 치환, 분리 기능 등을 제공한다
# 문자열을 분리하는 예로 하나 이상의 스페이스를 의미하는 '\s+'를 사용한다
import re

In [1030]:
text = 'foo    bar\t baz   \tqux'

In [1031]:
text

'foo    bar\t baz   \tqux'

In [1032]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

- re.split('\s+', text)를 사용하면 먼저 정규표현식이 컴파일되고 그 다음에 split 메서드가 실행
- re.compile을 통해 직접 정규표현식을 컴파일하고 그렇게 얻은 정규표현식 객체를 재사용하는 것도 가능

In [1033]:
regex = re.compile('\s+')

In [1034]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

- 정규표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메서드

In [1035]:
regex.findall(text)

['    ', '\t ', '   \t']

- 정규표현식 안에서 \ 문자가 이스케이스되는 것을 피하려면 raw 문자열 표기법으로 문제를 회피할 수 있다.
- r'C:\x'은 'c:\\\\x'와 동일

- 같은 정규표현식을 다른 문자열에도 적용해야 한다면 re.compile을 이용해서 정규표현식 객체를 만들어 사용하는 방법 추천. CPU 사용량 절약
- match와 search는 findall 메서드와 관련
- findall: 문자열에서 일치하는 모든 부분의 문자열을 찾아줌
- search: 패턴과 일치하는 첫 번째 존재를 반환
- match: 이보다 더 엄격해서 문자열의 시작 부분에서 일치하는 것만 찾아준다.

- 책에 마지막 re.I를 적용하는 부분이 생략되어 제대로 동작하지 않는다. regex 변수를 추가해 주어야 한다.

In [1036]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# 이 부분이 생략되어 있음
regex = re.compile(pattern, re.I) # re.IGNORECASE는 정규표현식이 대∙소문자를 가리지 않도록 한다.

In [1037]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

- search는 텍스트에서 첫 번째 이메일 주소만 찾아준다. 이 정규표현식에 대한 match 객체는 그 패턴이 문자열 안에서 위치하는 시작점과 끝점만을 알려준다.

In [1038]:
m = regex.search(text)

In [1039]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [1040]:
text[m.start():m.end()]

'dave@google.com'

- regex.match는 None을 반환. 왜냐하면 그 패턴이 문자열의 시작점에서부터 일치하는지 검사하기 때문

In [1041]:
# 이 문서의 처음은 Dave로 시작하기 때문에 regex와 일치하지 않음
print(regex.match(text))

None


- sub 메서드는 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열 반환

In [1042]:
print (regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



- 이메일 주소를 찾아서 동시에 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사의 3가지 컴포넌트로 나눠야 한다면 각 패턴을 괄호로 묶어준다.

In [1043]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [1044]:
regex = re.compile(pattern, flags=re.IGNORECASE)

- 이렇게 만든 regex 객체의 match 메서드를 이용하면 groups 메서드를 통해 각 패턴 컴포넌트의 튜플을 얻을 수 있음

In [1045]:
m = regex.match('wesm@bright.net')

In [1046]:
type(m)

_sre.SRE_Match

In [1047]:
m.groups()

('wesm', 'bright', 'net')

In [1048]:
#group(0)은 튜플의 전체를 반환한다.
m.group(0)

'wesm@bright.net'

In [1049]:
m.group(1)

'wesm'

In [1050]:
m.group(2)

'bright'

In [1051]:
m.group(3)

'net'

- 패턴에 그룹이 있다면 findall 메서드는 튜플의 모든 목록을 반환

In [1052]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

- sub 역시 마찬가지로 \1, \2 같은 특수한 기호를 사용해서 각 패턴의 그룹에 접근 가능

In [1053]:
print (regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



- 파이썬에서 할 수 있는 정규표현식은 많이 있지만 이 책의 범위를 넘기 때문에 생략
- 위에서 살펴본 이메일 주소 정규표현식의 매치 그룹에 다음처럼 이름을 줄 수도 있다.

In [1054]:
regex = re.compile(r'''
(?P<username>[A-Z0-9._%+-]+)
@
(?P<domain>[A-Z0-9.-]+)
\.
(?P<suffix>[A-Z]{2,4})''', flags=re.IGNORECASE | re.VERBOSE)

- 이 정규표현식으로 생성한 match 객체는 그룹 이름이 지정되어 편리한 사전형을 만들어 준다.

In [1055]:
m = regex.match('wesm@bright.net')

In [1056]:
m.groupdict()

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

#### 정규표현식 메서드

인자 | 설명
--- | ---
findall, finditer | 문자열에서 겹치지 않는, 발견된 모든 패턴을 반환한다. findall 메서드는 모든 패턴의 리스트를 바노한하고, finditer 메서드는 이터레이터를 통해 하나씩 반환한다.
match | 문자열의 시작점부터 패턴을 찾고 선택적으로 패턴 컴포넌트를 그룹으로 나눈다. 일치하는 패턴이 있다면 match 객체를 반환하고 그렇지 않을 경우 None을 반환한다.
search | 문자열에서 패턴과 일치하는 내용을 검색하고 match 객체를 반환한다. match 메서드와는 달리 시작부터 일치하는 내용만 찾지 않고 문자열 어디든 일치하는 내용이 있다면 반환한다.
split | 문자열에서 패턴과 일치하는 부분을 쪼갠다.
sub, subn | 문자열에서 일치하는 모든 패턴(sub) 혹은 처음 n개의 패턴(subn)을 대체 표현으로 치환한다. 대체 표현 문자열은 \1, \2, ... 같은 기호를 사용해서 매치 그룹의 요소를 참조한다.

### 7.4.3 pandas의 벡터화된 문자열 함수

- 데이터를 분석하려면 먼저 뒤죽박죽인 데이터의 문자열을 다듬고 정규화하는 정리 작업 필요
- 문자열을 담고 있는 칼럼에 누락된 값이 있다면 일을 더 복잡하게 만든다.

In [1057]:
data = {'Dave': 'dave@google.com', 
        'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 
        'Wes': np.nan}

In [1058]:
data = Series(data)

In [1059]:
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [1060]:
data.isnull() #값이 비어있는지 확인하는 isnull()함수

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

- 문자열과 정규표현식 메서드는 data.map을 사용해서 각 값에 적용(lambda 혹은 다른 함수를 넘겨서) 할 수 있지만 NA 값을 만나면 실패한다. 이런 문제에 대처하기 위해 Series에는 NA값을 건너뛰게 하는 간결한 문자열 처리 메서드 존재
- 이는 Series의 str 속성을 이용하는데, 예를 들면 각 이메일 주소가 'gmail'을 포함하고 있는지 str.contains를 이용해서 검사 가능

In [1061]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [1062]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

- IGNORECASE 같은 re 옵션과 함께 정규표현식도 사용할 수 있다.

In [1063]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

- 벡터화된 요소를 꺼내오는 몇 가지 방법이 있는데, str.get을 이용하거나 str 속성의 색인을 이용하는 방법

In [1064]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  if __name__ == '__main__':


In [1065]:
matches

Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

In [1066]:
matches.str.get(1)#get(index)로 해당 위치의 내용을 불러옴

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [1067]:
matches.str[0]#str[index]로도 내용을 불러올 수 있음

Dave      dave
Rob        rob
Steve    steve
Wes        NaN
dtype: object

- 문자열을 잘라낼 수 있다.

In [1068]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

#### [벡터화된 문자열 메서드](http://pandas.pydata.org/pandas-docs/stable/basics.html#vectorized-string-methods)

메서드 | 설명
--- | ---
cat | 선택적인 구분자와 함께 요소별로 문자열을 이어 붙인다.
contains | 문자열이 패턴이나 정규표현식을 포함하는지를 나타내는 불리언 배열을 반환한다.
count | 일치하는 패턴의 개수를 반환한다.
endswith, startswith | 각 요소에 대해 x.endswith(pattern), x.startswith(pattern)와 동일한 동작을 한다.
findall | 각 문자열에 대해 일치하는 패턴/정규표현식의 전체 목록을 구한다.
get | i번째 요소를 반환한다.
join | Series의 각 요소를 주어진 구분자로 연결한다.
len | 각 문자열의 길이를 구한다.
lower, upper | 대∙소문자로 변환한다. 각 요소에 대한 x.lower(), x.upper()와 같다.
match | 주어진 정규표현식으로 각 요소에 대한 re.match를 수행하여 일치하는 그룹을 리스트로 반환한다.
pad | 문자열의 좌, 우 혹은 양쪽에 공백을 추가한다.
center | pad(side='both')와 동일하다.
repeat | 값을 복사한다. 예를 들어 s.str.repeat(3)는 각 문자열에 대한 x*3과 동일하다.
replace | 패턴/정규표현식과 일치하는 내용을 다른 문자열로 치환한다.
slice | Series 안에 있는 각 문자열을 자른다.
split | 정규표현식 혹은 구분자로 문자열을 나눈다.
strip, rstrip, lstrip | 각 요소의 개행문자를 포함한 공백문자를 제거한다.

## 7.5 예제: 미국 농무부 음식 데이터베이스

- US Department of Argiculture는 음식의 영양소 정보 데이터베이스 제공
- 영국 해커인 Ashley Williams는 이 데이터베이스를 다음과 같은 JSON 형식으로 [웹사이트](http://ashleyw.co.uk/project/food-nutrient-database)에 제공

    {
      "id": 21441,
      "description": "KENTUCKY FRIED CHICKEN, Fried Chicken, EXTRA CRISPY, Wing, meat and skin with breading",
      "tags": ["KFC"],  
      "manufacturer": "Kentucky Fried Chicken",
      "group": "Fast Foods",
      "portions": [
        {
          "amount": 1,
          "unit": "wing, with skin",
          "grams": 68.0
        },

        ...
      ],
      "nutrients": [
        {
          "value": 20.8,
          "units": "g",
          "description": "Protein",
          "group": "Composition"
        },

        ...
      ]
    }

- 각 음식에는 숫자로 된 고유 ID와 영양소 및 제공량을 담고 있는 2가지 리스트
- 이 데이터 형식은 분석하기에 편하지 않으므로 좀 더 나은 형태로 데이터 형식 변경

In [1069]:
import json

In [1070]:
db = json.load(open('data/foods-2011-10-03.json'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/foods-2011-10-03.json'

In [1071]:
len(db)

6636

- db에 있는 각 엔트리는 한 가지 음식에 대한 모든 정보를 담고 있는 사전형
- 'nutrients'(영양소) 필드는 사전의 리스트이며 각 항목은 한 가지 영양소에 대한 정보를 담고 있다.

In [1072]:
# 각 항목에는 각 음식에 대한 정보를 담는다
db[0].keys()

dict_keys(['id', 'description', 'tags', 'manufacturer', 'group', 'portions', 'nutrients'])

In [1073]:
db[0]['nutrients'][0]

{'description': 'Protein',
 'group': 'Composition',
 'units': 'g',
 'value': 25.18}

In [1074]:
db[0]['nutrients'][1]

{'description': 'Total lipid (fat)',
 'group': 'Composition',
 'units': 'g',
 'value': 29.2}

In [1075]:
nutrients = DataFrame(db[0]['nutrients'])

In [1076]:
nutrients[:7]

,description,group,units,value
0,Protein,Composition,g,25.18
1,Total lipid (fat),Composition,g,29.20
2,"Carbohydrate, by difference",Composition,g,3.06
3,Ash,Other,g,3.28
4,Energy,Energy,kcal,376.00
5,Water,Composition,g,39.28
6,Energy,Energy,kJ,1573.00


In [1077]:
len(nutrients)

162

- 사전의 리스트를 DataFrame으로 바꿀 때 추출할 필드 목록을 지정해줄 수 있다. 우리는 음식의 이름과 그룹, id 그리고 제조사를 추출할 것이다.

In [1078]:
info_keys = ['description', 'group', 'id', 'manufacturer']

In [1079]:
info = DataFrame(db, columns=info_keys)

In [1080]:
info[:5]

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,


In [1081]:
len(info)

6636

In [1082]:
info

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


- value_counts 메서드를 이용해서 음식 그룹의 분포 확인

In [1083]:
pd.value_counts(info.group)[:10]

Vegetables and Vegetable Products    812
Beef Products                        618
Baked Products                       496
Breakfast Cereals                    403
Legumes and Legume Products          365
Fast Foods                           365
Lamb, Veal, and Game Products        345
Sweets                               341
Pork Products                        328
Fruits and Fruit Juices              328
Name: group, dtype: int64

- 모든 영양소 정보를 분석
- 먼저 좀 더 쉽게 분석을 하기 위해 각 음시그이 영양소 정보를 거대한 테이블 하나에 담아보자.
- 그러려면 사전에 몇 가지 과정을 거쳐야 함
- 먼저 음식의 영양소 리스트를 하나의 DataFrame으로 변환하고, 음식의 id를 위한 칼럼을 하나 추가
- 그리고 이 DataFrame을 리스트에 추가하낟. 마지막으로 이 리스트를 concat 메서드를 사용해서 하나로 합친다.

In [1084]:
nutrients = []

for rec in db:#data cleaning을 위해 영양소 리스트의 id를 같은 음식별로 묶는 함수 rec
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)
    
nutrients = pd.concat(nutrients, ignore_index=True) 
#concat함수를 이용해 기존의 nutrients에 id column을 새로 생성하여 붙임.

In [1085]:
db[0]['nutrients']

[{'description': 'Protein',
  'group': 'Composition',
  'units': 'g',
  'value': 25.18},
 {'description': 'Total lipid (fat)',
  'group': 'Composition',
  'units': 'g',
  'value': 29.2},
 {'description': 'Carbohydrate, by difference',
  'group': 'Composition',
  'units': 'g',
  'value': 3.06},
 {'description': 'Ash', 'group': 'Other', 'units': 'g', 'value': 3.28},
 {'description': 'Energy', 'group': 'Energy', 'units': 'kcal', 'value': 376.0},
 {'description': 'Water',
  'group': 'Composition',
  'units': 'g',
  'value': 39.28},
 {'description': 'Energy', 'group': 'Energy', 'units': 'kJ', 'value': 1573.0},
 {'description': 'Fiber, total dietary',
  'group': 'Composition',
  'units': 'g',
  'value': 0.0},
 {'description': 'Calcium, Ca',
  'group': 'Elements',
  'units': 'mg',
  'value': 673.0},
 {'description': 'Iron, Fe',
  'group': 'Elements',
  'units': 'mg',
  'value': 0.64},
 {'description': 'Magnesium, Mg',
  'group': 'Elements',
  'units': 'mg',
  'value': 22.0},
 {'description': 

In [1086]:
nutrients

,description,group,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


In [1087]:
nutrients[:5]

,description,group,units,value,id
0,Protein,Composition,g,25.18,1008
1,Total lipid (fat),Composition,g,29.20,1008
2,"Carbohydrate, by difference",Composition,g,3.06,1008
3,Ash,Other,g,3.28,1008
4,Energy,Energy,kcal,376.00,1008


- DataFrame에는 중복된 데이터가 있으므로 제거

In [1088]:
nutrients.duplicated().sum()#중복된 값 더함

14179

In [1089]:
nutrients = nutrients.drop_duplicates()#중복된 값을 제거

In [1090]:
nutrients.duplicated().sum()

0

In [1091]:
nutrients

,description,group,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


- 'group'과 'description'은 모두 DataFrame 객체이므로쉽게 알아볼 수 있도록 이름을 바꿔준다.

In [1092]:
col_mapping = {'description': 'food',
               'group' : 'fgroup'}

In [1093]:
#col_mapping에서 새로 정한 이름으로 info를 rename
info = info.rename(columns=col_mapping, copy=False)

In [1094]:
info

,food,fgroup,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


In [1095]:
col_mapping = {'description': 'nutrient',
               'group': 'nutgroup'}

In [1096]:
nutrients = nutrients.rename(columns=col_mapping, copy=False)

In [1097]:
nutrients

,nutrient,nutgroup,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


- info 객체를 nutrients 객체로 병합한다.
- info는 6636개, nutrients는 375176개인데 info에는 unique 한 id값만 들어가 있고 nutrients에는 모든 영양소가 들어가 있다.

In [1098]:
ndata = pd.merge(nutrients, info, on='id', how='outer')

In [1099]:
ndata

,nutrient,nutgroup,units,value,id,food,fgroup,manufacturer
0,Protein,Composition,g,25.180,1008,"Cheese, caraway",Dairy and Egg Products,
1,Total lipid (fat),Composition,g,29.200,1008,"Cheese, caraway",Dairy and Egg Products,
2,"Carbohydrate, by difference",Composition,g,3.060,1008,"Cheese, caraway",Dairy and Egg Products,
3,Ash,Other,g,3.280,1008,"Cheese, caraway",Dairy and Egg Products,
4,Energy,Energy,kcal,376.000,1008,"Cheese, caraway",Dairy and Egg Products,
5,Water,Composition,g,39.280,1008,"Cheese, caraway",Dairy and Egg Products,
6,Energy,Energy,kJ,1573.000,1008,"Cheese, caraway",Dairy and Egg Products,
7,"Fiber, total dietary",Composition,g,0.000,1008,"Cheese, caraway",Dairy and Egg Products,
8,"Calcium, Ca",Elements,mg,673.000,1008,"Cheese, caraway",Dairy and Egg Products,
9,"Iron, Fe",Elements,mg,0.640,1008,"Cheese, caraway",Dairy and Egg Products,


In [1100]:
ndata[:5]

,nutrient,nutgroup,units,value,id,food,fgroup,manufacturer
0,Protein,Composition,g,25.18,1008,"Cheese, caraway",Dairy and Egg Products,
1,Total lipid (fat),Composition,g,29.20,1008,"Cheese, caraway",Dairy and Egg Products,
2,"Carbohydrate, by difference",Composition,g,3.06,1008,"Cheese, caraway",Dairy and Egg Products,
3,Ash,Other,g,3.28,1008,"Cheese, caraway",Dairy and Egg Products,
4,Energy,Energy,kcal,376.00,1008,"Cheese, caraway",Dairy and Egg Products,


In [1101]:
ndata.ix[30000]

nutrient                                       Glycine
nutgroup                                   Amino Acids
units                                                g
value                                             0.04
id                                                6158
food            Soup, tomato bisque, canned, condensed
fgroup                      Soups, Sauces, and Gravies
manufacturer                                          
Name: 30000, dtype: object

In [1102]:
#result에 영양소정보와 음식 분류를 그룹화한 것을 넣는다.
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)

In [1103]:
result

nutrient          fgroup                           
Adjusted Protein  Sweets                               12.9000
                  Vegetables and Vegetable Products     2.1800
Alanine           Baby Foods                            0.0850
                  Baked Products                        0.2480
                  Beef Products                         1.5500
                  Beverages                             0.0030
                  Breakfast Cereals                     0.3110
                  Cereal Grains and Pasta               0.3730
                  Dairy and Egg Products                0.2710
                  Ethnic Foods                          1.2900
                  Fast Foods                            0.5140
                  Fats and Oils                         0.0000
                  Finfish and Shellfish Products        1.2180
                  Fruits and Fruit Juices               0.0270
                  Lamb, Veal, and Game Products         1.4080
   

- 좀 더 응용하면 각 영양소가 어떤 음식에 가장 많이 들었는지 찾아볼 수도 있다.

In [1104]:
by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

get_maximum = lambda x: x.xs(x.value.idxmax())#음식별 영양소 함량이 가장 높은 것을 골라내기 위한 함수
get_minimum = lambda x: x.xs(x.value.idxmin())#음식별 영양소 함량이 가장 낮은 것을 골라내기 위한 함수

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]

max_foods.food = max_foods.food.str[:50]

In [1105]:
max_foods

value  \
nutgroup    nutrient                                     
Amino Acids Alanine                              8.009   
            Arginine                             7.436   
            Aspartic acid                       10.203   
            Cystine                              1.307   
            Glutamic acid                       17.452   
            Glycine                             19.049   
            Histidine                            2.999   
            Hydroxyproline                       0.803   
            Isoleucine                           4.300   
            Leucine                              7.200   
            Lysine                               6.690   
            Methionine                           1.859   
            Phenylalanine                        4.600   
            Proline                             12.295   
            Serine                               4.600   
            Threonine                            3.300   
            Tryptophan                           1.600   
            Tyrosine                             3.300   
            Valine                               4.500   
Composition Adjusted Protein                    12.900   
            Carbohydrate, by difference        100.000   
            Fiber, total dietary                79.000   
            Protein                             88.320   
            Sugars, total                       99.800   
            Total lipid (fat)                  100.000   
            Water                              100.000   
Elements    Calcium, Ca                       7364.000   
            Copper, Cu                          15.050   
            Fluoride, F                        584.000   
            Iron, Fe                            87.470   
...                                                ...   
Vitamins    Cryptoxanthin, beta               6186.000   
            Dihydrophylloquinone               103.800   
            Folate, DFE                       2630.000   
            Folate, food                      2340.000   
            Folate, total                     2340.000   
            Folic acid                        1538.000   
            Lutein + zeaxanthin              39550.000   
            Lycopene                         46260.000   
            Menaquinone-4                       33.200   
            Niacin                              97.000   
            Pantothenic acid                    35.000   
            Retinol                          30000.000   
            Riboflavin                          14.300   
            Thiamin                             20.000   
            Tocopherol, beta                     6.490   
            Tocopherol, delta                   30.880   
            Tocopherol, gamma                  100.880   
            Vitamin A, IU                   100000.000   
            Vitamin A, RAE                   30000.000   
            Vitamin B-12                        98.890   
            Vitamin B-12, added                 24.000   
            Vitamin B-6                         12.000   
            Vitamin C, total ascorbic acid    2400.000   
            Vitamin D                        10000.000   
            Vitamin D (D2 + D3)                250.000   
            Vitamin D2 (ergocalciferol)         28.100   
            Vitamin D3 (cholecalciferol)        27.400   
            Vitamin E (alpha-tocopherol)       149.400   
            Vitamin E, added                    46.550   
            Vitamin K (phylloquinone)         1714.500   

                                                                                         food  
nutgroup    nutrient                                                                           
Amino Acids Alanine                                         Gelatins, dry powder, unsweetened  
            Arginine                                             Seeds, sesame flour, low-fat  
          

In [1106]:
#아미노산이 들어있는 음식 표시
max_foods.ix['Amino Acids']['food']

nutrient
Alanine                           Gelatins, dry powder, unsweetened
Arginine                               Seeds, sesame flour, low-fat
Aspartic acid                                   Soy protein isolate
Cystine                Seeds, cottonseed flour, low fat (glandless)
Glutamic acid                                   Soy protein isolate
Glycine                           Gelatins, dry powder, unsweetened
Histidine                Whale, beluga, meat, dried (Alaska Native)
Hydroxyproline    KENTUCKY FRIED CHICKEN, Fried Chicken, ORIGINA...
Isoleucine        Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Leucine           Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Lysine            Seal, bearded (Oogruk), meat, dried (Alaska Na...
Methionine                    Fish, cod, Atlantic, dried and salted
Phenylalanine     Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Proline                           Gelatins, dry powder, unsweetened
Serine            Soy protein isolate, 

## [About my IPython in github](https://github.com/re4lfl0w/ipython)